In [46]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

%matplotlib inline

# Import dataset 

In [2]:
df = pd.read_csv("Customer-Churn.csv")
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


# Missing values

In [5]:
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [3]:
# Dropping columns with missing values 
df.dropna(inplace=True)

# Splitting data

In [120]:
#Separate features and label

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Churn']), df["Churn"], 
                                                    train_size = 0.7, stratify = df["Churn"] )

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4922, 19) (4922,)
(2110, 19) (2110,)


# Preprocessing 

In [122]:
#preprocessor for feature matrix X
preprocessor = ColumnTransformer([('scaler', StandardScaler(), make_column_selector(dtype_include = 'number')),
                                 ('encoder', OneHotEncoder(drop = 'first'), make_column_selector(dtype_include = 'object'))])

In [123]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [143]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4922, 30) (4922,)
(2110, 30) (2110,)


# Model Selection 

Testing different baseline models

In [125]:
# Iterate through the groups of model to find the model with higest accuracy
# using 5 fold cross validation
for model in [GaussianNB(),KNeighborsClassifier(),GradientBoostingClassifier(), RandomForestClassifier(),SVC()]:
    cv_score = cross_val_score(model, X_train, y_train, scoring='accuracy' ,cv=5)
    print("Accuracy score for %s: %.3f"%(model.__class__.__name__, cv_score.mean()))

Accuracy score for GaussianNB: 0.659
Accuracy score for KNeighborsClassifier: 0.770
Accuracy score for GradientBoostingClassifier: 0.798
Accuracy score for RandomForestClassifier: 0.791
Accuracy score for SVC: 0.800


Cross validation score indicates that SVC and GradientBoostinglassifier are the best performing models 

# Model Optimization

In this part, I will optimize the model by appluing both Feature Selection and Hyperparameter tunning to both algorithms

## Support Vector Machine 

In [136]:
svm = Pipeline([("fs",SelectKBest(f_classif)),
               ("svc",SVC(kernel='rbf'))])

# setting possible values for the penalty parameter of Support Vector Classifier() and Select Percentile
svm_param = [{'fs__k':np.arange(25,30,1),
              'svc__C':np.arange(0,1,0.1),
              'svc__gamma':np.arange(0,0.4,0.05)}]

svm_tune = GridSearchCV(estimator = svm, param_grid = svm_param,
                       scoring='accuracy',n_jobs=-1, cv=5,
                       return_train_score=True)
# Fit the training data into it,
#It will determine which parameter 
#returns the best model using cross validation
svm_tune.fit(X_train,y_train)

# Get the best hyper parameters for the model
svm_tune.best_params_

{'fs__k': 28, 'svc__C': 0.4, 'svc__gamma': 0.15000000000000002}

In [137]:
# Get the best models from GridSearch
svm_model = svm_tune.best_estimator_

svm_model.fit(X_train, y_train)

Pipeline(steps=[('fs', SelectKBest(k=28)),
                ('svc', SVC(C=0.4, gamma=0.15000000000000002))])

___

## GradientBoostinglassifier

In [142]:
# Building Gradient Boosting Model using SelectKBest() and GradientBoostingClassifier()
gbm = Pipeline([("fs",SelectKBest(f_classif)),
                ("gbc",GradientBoostingClassifier())])

gbm_param_grid = [{'fs__k':np.arange(25,30,1),
                   'gbc__loss':['deviance', 'exponential'],
                   'gbc__learning_rate':np.arange(0,0.2,0.01),
                   'gbc__n_estimators':np.arange(50,200,50)}]

gbm_tune = GridSearchCV(gbm, gbm_param_grid,cv=5,
                        scoring='accuracy',n_jobs=-1,
                        return_train_score=True)

gbm_tune.fit(X_train,y_train)

gbm_tune.best_params_

{'fs__k': 29,
 'gbc__learning_rate': 0.14,
 'gbc__loss': 'exponential',
 'gbc__n_estimators': 50}

In [144]:
gbm_model = gbm_tune.best_estimator_

gbm_model.fit(X_train, y_train)

Pipeline(steps=[('fs', SelectKBest(k=29)),
                ('gbc',
                 GradientBoostingClassifier(learning_rate=0.14,
                                            loss='exponential',
                                            n_estimators=50))])

# Evaluate Classification model 

## Support Vector Classifier

In [138]:
#fitting training set to data
y_predicted = svm_model.predict(X_test)

In [139]:
accuracy_score(y_true= y_test, y_pred= y_predicted)

0.7924170616113744

In [140]:
confusion_matrix(y_true = y_test,y_pred = y_predicted)

array([[1424,  125],
       [ 313,  248]])

In [141]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

          No       0.82      0.92      0.87      1549
         Yes       0.66      0.44      0.53       561

    accuracy                           0.79      2110
   macro avg       0.74      0.68      0.70      2110
weighted avg       0.78      0.79      0.78      2110



## Gradient Boosting Classifier 

In [145]:
y_predicted = gbm_model.predict(X_test)

In [146]:
accuracy_score(y_true= y_test, y_pred= y_predicted)

0.8042654028436019

In [147]:
confusion_matrix(y_true = y_test,y_pred = y_predicted)

array([[1417,  132],
       [ 281,  280]])

In [148]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

          No       0.83      0.91      0.87      1549
         Yes       0.68      0.50      0.58       561

    accuracy                           0.80      2110
   macro avg       0.76      0.71      0.72      2110
weighted avg       0.79      0.80      0.79      2110

